# gurobiによる制約最適化を行う

In [5]:
import math
import random
import networkx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import networkx
from gurobipy import *

## 変数
- $x_{ik}$ $x_{jl}$ : 人iが避難施設kに避難した場合に１　バイナリー変数

- $D_{kl}$ : 避難施設kとlの間に定義される伝染危険率　近いと大きい

- $W_{ij} $ : 人i,jでの伝染危険率
    - 感染の差の自乗とmax(jの感染率 – iの感染率,0)・iの死亡率 + max(iの感染率 – jの感染率,0)・ jの死亡率 の重み付き和

## 定数

- max(jの感染率 – iの感染率,0)・iの死亡率 + max(iの感染率 – jの感染率,0)・ jの死亡率  

- 感染率の差の自乗和

## 目的関数
-  min $W_{ij} + D_{kl} + x_{ik} + x_{jl}$


## 制約
- すべての人 iに対して，x[i,k]の施設kに対する合計 = 1   

- 施設kに対してx[i]=kの人に対する合計が施設kの容量以下

## 必要な集合
- 人の集合 people
- 人の感染率の集合 infec_rate
- 人の死亡率の集合 mort_rate
- 施設集合 shelters
- 施設の位置情報 locations
- 施設の容量 capacity

In [55]:
random.seed(0)

In [56]:
N = 10 #人の数
people = [i for i in range(N)] #人の集合
infec_rate,mort_rate = {},{}
for i in people:
    infec_rate[i] = random.randint(0,100)/100
    mort_rate[i] = random.randint(0,100)/100
print(infec_rate)
print(mort_rate)

{0: 0.49, 1: 0.53, 2: 0.33, 3: 0.62, 4: 1.0, 5: 0.61, 6: 0.74, 7: 0.64, 8: 0.36, 9: 0.96}
{0: 0.97, 1: 0.05, 2: 0.65, 3: 0.51, 4: 0.38, 5: 0.45, 6: 0.27, 7: 0.17, 8: 0.17, 9: 0.12}


In [57]:
N_s = 5
shelters = [i for i in range(N_s)] #施設の集合
locations,capacity = {},{}
for i in people:
    locations[i] = np.random.rand(2) * 100
    capacity[i] = random.randint(0,5)
print(locations)
print(capacity)

{0: array([43.59621865, 58.23605832]), 1: array([93.74454084, 82.3964464 ]), 2: array([21.77836202, 44.72717208]), 3: array([38.8275176 , 90.37562388]), 4: array([ 2.48258343, 96.86650974]), 5: array([86.01523857, 18.87588913]), 6: array([10.63862211, 73.17060018]), 7: array([25.50870529, 95.13083882]), 8: array([5.89369258, 5.02577638]), 9: array([90.84319575,  3.50343052])}
{0: 4, 1: 2, 2: 4, 3: 5, 4: 4, 5: 1, 6: 2, 7: 0, 8: 5, 9: 0}


In [58]:
model = Model()

In [87]:
#人iが避難施設kに避難した場合に1になる0-1変数 x[i,k] x[j,l]
x = {}
for i in people:
    for k in shelters:
        x[i,k] = model.addVar(ub=1, vtype="B", name="x(%s,%s)"%(i,k))
        
for j in people:
    for l in shelters:
        x[j,l] = model.addVar(ub=1, vtype="B", name="x(%s,%s)"%(j,l))
        
model.update()

In [88]:
# D[k,l]
D = {}
for k in shelters:
    for l in shelters:
        D[k,l] = model.addVar(ub=1,lb=0, vtype = 'C', name="D(%s,%s)"%(k,l))
model.update()
## 変数じゃなくて定数じゃね


In [89]:
# W[i,j]
W = {}
for i in people:
    for j in people:
        w1 = max(infec_rate[j] - infec_rate[i],0) * mort_rate[i] + max(infec_rate[i] - infec_rate[j],0) * mort_rate[j]
        w2 = (infec_rate[j] - infec_rate[i]) ** 2
        W[i,j] = model.addVar(ub=1,lb=0, vtype = 'C', name="W(%s,%s)"%(i,j))
        W[i,j]*w1*w2
model.update()

## 制約

In [94]:
# すべての人 iに対して，x[i,k]の施設kに対する合計 = 1   
for i in people:
    model.addConstr(quicksum(x[i,k] for k in shelters)== 1)
    
#すべての施設kに対して，x[i,k] の人に対する合計が施設kの容量以下
for k in shelters:
    model.addConstr(quicksum(x[i,k] for i in people) <= capacity[k])
model.update()

0
1
2
3
4


In [98]:
#目的関数
model.setObjective(quicksum(W[i,j]*D[k,l]*x[i,k]*x[j,l]
                            for i in people
                            for j in people
                            for k in shelters
                            for l in shelters
                            if j>i), GRB.MINIMIZE)

model.update()

GurobiError: Invalid argument to QuadExpr multiplication